<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module2/M2_2/5_TheSimplexMethod_Minimization2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Simplex Method: Minimization 2D
**OPIM 5641: Business Decision Modeling - University of Connecticut**

Material from: "Elementary Linear Algebra" - 8th Edition (Ron Larson) - Chapter 9.

---------------------------------------------------------------------------
**Objectives:**
* Determine the dual of a linear programming problem that  minimizes an objective function.  
* Use the simplex method to solve a linear programming problem that minimizes an objective function

In [ ]:
# import modules
# Matrix makes a sympy Matrix, Rational is the same Fraction from fraction, pprint makes it pretty print, nsimplify converts decimals to fractions (rational numbers)
from sympy import Matrix, Rational, pprint, nsimplify 

# standard modules
import numpy as np
import pandas as pd

**Background:** You previously applied the simplex method to linear programming problems in **standard form** where the objective function was to be **maximized** (all constraints had $\leq$ symbols). In this section, you will extend this procedure to linear programming problems in which the objective function is to be **minimized** (requires all constraints to have $\geq$ symbols). 

Specifically, a minimization problem is defined in standard form when the objective function:

$w = c_1x_1 + c_2x_2 + ... c_nx_n$

is to be minimized, subject to the constraints

$a_m$$_1x_1 + $ $a_m$$_2x_2 + $ $...+ a_m$$_nx_n \geq b_m$

where $x_i \geq 0$ and $b_i \geq 0$.

The basic procedure used to solve such a problem is to convert it to a **maximization problem** in standard form, then apply the Simplex method. At the end, you just need to interpret/read off the final output (tableau) a little differently.

Let's consider a simple problem in 2D.


#Problem Description
Find the minimum value of
$w = 0.12x_1 + 0.15x_2$

subject to:
* $60x_1 + 60x_2 \geq 300$
* $12x_1 + 6x_2 \geq 36$
* $10x_1 + 30x_2 \geq 90$
* $x_1,x_2 \geq 0$

First of all - notice that $z$ is nowhere to be found - as a naming convention, we will use $w$ for minimization problems to help keep us organized.

To solve this problem using simplex, you must convert it to a maximization problem. The first step is to form the augmented matrix for this system of inequalities. To this augmented matrix, add a last row that represents the coefficients of the objective function.

Note that we have not rearranged terms for the objective function. The values of each of the decision variables are still the same.

In our example, $A$ will always be the matrix, and `tmp` will always be a temporary copy of $A$ that we will color code. See helper function at the end to color a pandas dataframe, rename rows and columns etc.



In [ ]:
# this function is useful for coloring a single cell
# you'll see how to use it later

# Custom function to color the desired cell
def styling_specific_cell(x,row_idx,col_idx):
    color = 'background-color: yellow; color: red'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler

## Set up initial matrix ($A$)

In [ ]:
# form the matrix
A = Matrix([[60, 60, 300],
              [12, 6, 36],
              [10, 30, 90],
              [0.12, 0.15, 0]])

# show the matrix
pprint(A)

⎡ 60    60   300⎤
⎢               ⎥
⎢ 12    6    36 ⎥
⎢               ⎥
⎢ 10    30   90 ⎥
⎢               ⎥
⎣0.12  0.15   0 ⎦


In [ ]:
# if you want to make it pretty, you can use pandas
# example
# link: https://stackoverflow.com/questions/40335140/how-to-highlight-both-a-row-and-a-column-at-once-in-pandas

# make sure you convert A to a numpy array for easy viewing - and make it a float 
# so that stuff isn't stored as a string
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['x1', 'x2', 'b']
tmp.index=['R0', 'R1', 'R2', 'R3']
tmp

# w is in the bottom right cell

# style - this is for an individual cell
idx_c = 2   # Column index of cell to color 
idx_r = 3   # Row index of cell to color

# see the columns 's1' and 's2' called out in the code below?
tmp.style\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,x1,x2,b
R0,60.000000,60.000000,300.000000
R1,12.000000,6.000000,36.000000
R2,10.000000,30.000000,90.000000
R3,0.120000,0.150000,0.000000


## Transpose ($A$)

Now, take the transpose of this matrix. Notice how each column becomes a row - stare at this for a moment if it is new to you.

Also note that we have not added any slack variables yet.

In [ ]:
# transpose
# we will keep overwriting A
A = A.T
pprint(A)

⎡60   12  10  0.12⎤
⎢                 ⎥
⎢60   6   30  0.15⎥
⎢                 ⎥
⎣300  36  90   0  ⎦


When we take the transpose, we change our variables from $x$ to $y$.

In [ ]:
# you can make it pretty with pandas
# we have not added slack variables yet (coming soon)
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

,y1,y2,y3,b
R0,60.0,12.0,10.0,0.12
R1,60.0,6.0,30.0,0.15
R2,300.0,36.0,90.0,0.00


Great! And now we will interpret this transposed matrix as a maximization problem. Will add our slack variables in a moment.

# Dual Maximization Problem
In order to interpret the transposed matrix as a maximization problem, we introduce new variables $y_1, y_2, y_3$. This corresponding maximization problem is called the dual of the original problem.

So, now we need to find the maximum value of:

$z = 300y_1 + 36y_2 + 90y_3$

subject to:
* $60y_1 + 12y_2 + 10y_3 + s_1 \leq 0.12$
* $60y_1 + 6y_2 + 30y_3 + s_2 \leq 0.15$
* $y_1,y_2,y_3 \geq 0$

Do you see how all of the $\geq$ constraints were switched to $\leq$ (does not include nonnegativity, which is always true)? You still need to put this in standard form and set up the initial simplex tableau.

Notice our naming conventions here - $w$ has been replaced by $z$, and $x$ has been replaced by $y$. This naming convention will help keep you organized!

The solution of the original minimization problem can be found by applying the simplex method to the new dual problem. At the end, you'll just need to interpret the final tableau a little differently.

Let's remake A but now we will include our slack variables $s_1$ and $s_2$.

In [ ]:
# since this is a maximization problem
# we need to remake our initial tableau and add slack variables


# column names are y1, y2, y3, s1, s2, b
A = Matrix([[60, 12, 10, 1, 0, 0.12],
            [60, 6, 30, 0, 1, 0.15],
            [-300, -36, -90, 0, 0, 0]])

pprint(A)

⎡ 60   12   10   1  0  0.12⎤
⎢                          ⎥
⎢ 60    6   30   0  1  0.15⎥
⎢                          ⎥
⎣-300  -36  -90  0  0   0  ⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float)) # you only need this in the first example
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# style - this is for an individual cell
idx_r = 2   # Row index of cell to color
idx_c = 5   # Column index of cell to color 

# see the columns 's1' and 's2' called out in the code below?
# see how I am apply the highlighting of columns twice?
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 's1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 's2' else '' for i in x])\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,b
R0,60.000000,12.000000,10.000000,1.000000,0.000000,0.120000
R1,60.000000,6.000000,30.000000,0.000000,1.000000,0.150000
R2,-300.000000,-36.000000,-90.000000,0.000000,0.000000,0.000000


Do you see how we have incorporated our slack variabes $s_1$ and $s_2$ (blue- they correspond to values of 0.12 and 0.15 - this is the 'do nothing' solution and we get a $z$ of 0)? And do you see how we have negative values in the bottom row? $z$ is in the bottom right (yellow/red cell). 

The **basic variables** are s1 and s2 and this results in a $z$ of 0. We can do better than that!

Let's move onto pivoting...

# Reminder on Pivoting

Remember - after you have set up the initial simplex tableau for a linear programming problem, the simplex method consists of checking for optimality and then, when the current solution is not optimal, improving the current solution. (An improved solution is one that has a larger z-value than the current solution.) 

To improve the current solution, bring a new basic variable into the solution, the entering variable. This implies that one of the current basic variables (the departing variable) must leave, otherwise you would have too many variables for a basic solution. Choose the entering and departing variables as listed below. 

1. The **entering variable** corresponds to the least (the most negative) entry in the bottom row of the tableau, excluding the “b-column.” 
2.  The **departing variable** corresponds to the least nonnegative ratio $b_i/$$a_i$$_j$ in the column determined by the entering variable, when $a_i$$_j$ $> 0$.
3.  The entry in the simplex tableau in the entering variable’s column and the departing variable’s row is the **pivot**. 

Finally, to form the improved solution, apply Gauss-Jordan elimination (https://online.stat.psu.edu/statprogram/reviews/matrix-algebra/gauss-jordan-elimination) to the column that contains the pivot, as illustrated in Example 1.
 

# Pivot #1
As a refresher, here is the initial simplex tableau. 

In [ ]:
tmp

,y1,y2,y3,s1,s2,b
R0,60.0,12.0,10.0,1.0,0.0,0.12
R1,60.0,6.0,30.0,0.0,1.0,0.15
R2,-300.0,-36.0,-90.0,0.0,0.0,0.00


## Entering Variable
Step one is to look for the most negative value in the initial tableau (the **entering variable**), which is -300 ($y_1$).

In [ ]:
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])

,y1,y2,y3,s1,s2,b
R0,60.000000,12.000000,10.000000,1.000000,0.000000,0.120000
R1,60.000000,6.000000,30.000000,0.000000,1.000000,0.150000
R2,-300.000000,-36.000000,-90.000000,0.000000,0.000000,0.000000


## Departing Variable

Then, compute ratios $b_i/$$a_i$$_j$ to determine which variable is the **departing variable**.

$0.12/60 = 0.002$ `s1`

$0.15/60 = 0.0025$ `s2`

Since $s_1$ is the *smaller positive number*, it becomes the **departing variable**.

In [ ]:
print(0.12/60) # s1 (winner! barely...)
print(0.15/60) # s2 

0.002
0.0025


Now that means the pivot element is '60' in the top left corner, which we will need to turn into a 1. To do this, we multiply Row 0 (R0) by 1/60*R0

In [ ]:
# before
pprint(A)

⎡ 60   12   10   1  0  0.12⎤
⎢                          ⎥
⎢ 60    6   30   0  1  0.15⎥
⎢                          ⎥
⎣-300  -36  -90  0  0   0  ⎦


In [ ]:
# before - make it pretty
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# highlight the pivot element
idx_r = 0
idx_c = 0

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,b
R0,60.000000,12.000000,10.000000,1.000000,0.000000,0.120000
R1,60.000000,6.000000,30.000000,0.000000,1.000000,0.150000
R2,-300.000000,-36.000000,-90.000000,0.000000,0.000000,0.000000


## Turn Pivot Element Into a '1'.

Notice how we are using `Rational` here instead of `Fraction`... you will see why in a moment! In the future, you can just use `Rational` and the `nsimplify` code, it won't hurt.

In [ ]:
# after
A[0,:] = Rational(1,60)*A[0,:]
pprint(A) # check your work

⎡ 1    1/5  1/6  1/60  0  0.002⎤
⎢                              ⎥
⎢ 60    6   30    0    1  0.15 ⎥
⎢                              ⎥
⎣-300  -36  -90   0    0    0  ⎦


In [ ]:
# if it looks a little messy, try nsimplify
# make sure you wrap it in a Matrix()...
# otherwise you will get an 'immutable matrix' error
A = Matrix(nsimplify(A, rational=True))
pprint(A)

⎡ 1    1/5  1/6  1/60  0  1/500⎤
⎢                              ⎥
⎢ 60    6   30    0    1  3/20 ⎥
⎢                              ⎥
⎣-300  -36  -90   0    0    0  ⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# highlight the pivot element
idx_r = 0
idx_c = 0

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,b
R0,1,1/5,1/6,1/60,0,1/500
R1,60,6,30,0,1,3/20
R2,-300,-36,-90,0,0,0


## Use GJ elimination and turn all other values above and below pivot element into 0s.

Great! Now we need to get rid of the $60$ and $-300$ that are below the pivot element.

Leave the row of interest ($R1$ and $R2$) as-is, and then add multiples of $R0$ to it.

## R1

In [ ]:
# R1 = R1 -60*R0
A[1,:] = A[1,:] - 60*A[0,:]


## R2

In [ ]:
# R2 = R2 + 300*R0
A[2,:] = A[2,:] + 300*A[0,:]

In [ ]:
# check your work
pprint(A)

⎡1  1/5  1/6  1/60  0  1/500⎤
⎢                           ⎥
⎢0  -6   20    -1   1  3/100⎥
⎢                           ⎥
⎣0  24   -40   5    0   3/5 ⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# highlight the pivot element
idx_r = 0
idx_c = 0

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,b
R0,1,1/5,1/6,1/60,0,1/500
R1,0,-6,20,-1,1,3/100
R2,0,24,-40,5,0,3/5


## Check your work - feasible solution?
Yes, but we will read it off later.

### Any negative numbers in bottom row?
Yes, so we need to keep going.

# Pivot #2



## Entering Variable
OK! Check and see if there are negative elements. We see a -40 in the bottom row, so $y_3$ will become our **entering variable**.



In [ ]:
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y3' else '' for i in x])

,y1,y2,y3,s1,s2,b
R0,1,1/5,1/6,1/60,0,1/500
R1,0,-6,20,-1,1,3/100
R2,0,24,-40,5,0,3/5


## Departing Variable
Our **departing variable** is going to be b/a... we choose the SMALLEST positive number. 

$(1/500) / (1/6)= 0.012$ `(R0)`

$(3/100) / 20 = 0.0015$ `(R1)`

In [ ]:
print((1/500)/(1/6)) #R0
print((3/100)/20) # R1 is the winner! smallest, POSITIVE number.


0.012
0.0015


In [ ]:
# let's show the new entering and departing variables
# make it pretty

tmp = pd.DataFrame(np.array(A))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# highlight the pivot element
idx_r = 1
idx_c = 2

# apply style to rows and columns
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y3' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R1' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,b
R0,1,1/5,1/6,1/60,0,1/500
R1,0,-6,20,-1,1,3/100
R2,0,24,-40,5,0,3/5


## Turn Pivot Element Into a '1'

And so we see that the 20 is going to be our pivot element. Let's turn this into a '1'.

In [ ]:
A[1,:] = Rational(1,20)*A[1,:]
pprint(A)

⎡1   1/5   1/6  1/60    0    1/500 ⎤
⎢                                  ⎥
⎢0  -3/10   1   -1/20  1/20  3/2000⎥
⎢                                  ⎥
⎣0   24    -40    5     0     3/5  ⎦


## Use GJ elimination and turn all other values above and below pivot element into 0s.

### R0

In [ ]:
# looks good! now we need to get rid of the
# 1/6 in R0
# -40 in R2

# remember, leave the row of interest as is
# and add multiples of the row with the pivot element

# fix R0
A[0,:] = A[0,:] - Rational(1,6)*A[1,:]


### R2

In [ ]:
# fix R2
A[2,:] = A[2,:] + 40*A[1,:]

# remember our titles
pprint(A) # WOOF! let's make it look nicer

⎡1   1/4   0  1/40   -1/120  7/4000⎤
⎢                                  ⎥
⎢0  -3/10  1  -1/20   1/20   3/2000⎥
⎢                                  ⎥
⎢                              33  ⎥
⎢0   12    0    3      2       ──  ⎥
⎣                              50  ⎦


Let's take a look at what we did. Are there any negative values in the bottom row... nope! You are done.

In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# highlight the pivot element
idx_r = 1
idx_c = 2

# apply style to rows and columns
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y3' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R1' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,b
R0,1,1/4,0,1/40,-1/120,7/4000
R1,0,-3/10,1,-1/20,1/20,3/2000
R2,0,12,0,3,2,33/50


# Reading the tableau

So, do you see any negative pivot elements in the bottom? NOPE! **So, you are done.**

Time to read off the solution (be careful - it's different now!)

The solutional of the dual maximization problem is $z = 33/50$. The x values are the slack variables, $x_1 = s_1 = 3$ and $x_2 = s_2 = 2$.

And so we have a new theorem: the von Neumann Duality principle.

The objective value $w$ of a minimization problem in standard form has a  minimum value if and only if the objective value $z$ of the dual maximization problem has a maximum value. Moreover, the minimum value of $w$ is equal to the maximum value of $z$.

Recall our original objective function:
$w = 0.12x_1 + 0.15x_2$

# Appendix: Color coding a table
Use this script to update the colors in your tables. You can use this all over the place.

In [ ]:
# a random sample array

A = Matrix([[60,   12,   10,   1,  0,  0.12],
            [60,    6,   30,   0,  1,  0.15],                      
            [-300,  -36,  -90,  0,  0,   0]])

# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float)) # you only need this in the first example
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# add some color to highlight s1 and s2 and z
# Custom function to color the desired cell
def styling_specific_cell(x,row_idx,col_idx):
    color = 'background-color: yellow; color: red'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler

# highlight a single cell pivot element
idx_r = 0
idx_c = 0

# apply style to rows and columns
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,b
R0,60.000000,12.000000,10.000000,1.000000,0.000000,0.120000
R1,60.000000,6.000000,30.000000,0.000000,1.000000,0.150000
R2,-300.000000,-36.000000,-90.000000,0.000000,0.000000,0.000000
